# **Cargado de archivos**

In [1]:
# =============================================================================
# Configuración optimizada de la sesión de Spark
# =============================================================================
# Esta configuración es más robusta para conjuntos de datos grandes.
# Debe ejecutarse una vez al inicio del notebook.

from pyspark.sql import SparkSession
import os

spark = SparkSession.builder \
    .appName("BlueBikes-Project") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.network.timeout", "800s") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.driver.maxResultSize", "2g") \
    .getOrCreate()

# Establecer variables de entorno para que PySpark funcione correctamente en algunos entornos
import sys
# os.environ["PYSPARK_PYTHON"] = sys.executable
# os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable
os.environ["PYSPARK_PYTHON"] = "python"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python"

In [2]:
# =============================================================================
# Cargar DataFrame procesado desde Google Drive (para Colab)
# =============================================================================
# Esta celda activa tu Google Drive y lee los datos procesados del archivo Parquet almacenado allí.
# Usarlo en el entorno de Google Colab.

from google.colab import drive

print("Attempting to mount Google Drive...")
try:
    drive.mount('/content/drive')

    gdrive_path = "/content/drive/MyDrive/DM_PRJ/df_final_bluebikes.parquet"
    print(f"Cargando datos desde la ruta de Google Drive: {gdrive_path}")

    # Spark leerá la carpeta Parquet directamente
    df_final = spark.read.parquet(gdrive_path)

    print("✅ DataFrame cargado exitosamente desde Google Drive.")

    # Verify the schema and show a few rows
    print("Esquema de DataFrame:")
    df_final.printSchema()

    print("Muestra de los datos cargados:")
    df_final.show(5, truncate=False)

except Exception as e:
    print(f"❌ Error al cargar datos desde Google Drive. Asegúrate de que el archivo exista en '{gdrive_path}'.")
    print(f"Detalles del error: {e}")

Attempting to mount Google Drive...
Mounted at /content/drive
Cargando datos desde la ruta de Google Drive: /content/drive/MyDrive/DM_PRJ/df_final_bluebikes.parquet
✅ DataFrame cargado exitosamente desde Google Drive.
Esquema de DataFrame:
root
 |-- start_station_id: string (nullable = true)
 |-- ride_id: string (nullable = true)
 |-- started_at: timestamp (nullable = true)
 |-- ended_at: timestamp (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_lat: double (nullable = true)
 |-- start_lng: double (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_lat: double (nullable = true)
 |-- end_lng: double (nullable = true)
 |-- member_casual: string (nullable = true)
 |-- duration_sec: long (nullable = true)
 |-- schema_version: string (nullable = true)
 |-- periodo: string (nullable = true)
 |-- log_duration: double (nullable = true)
 |-- trip_year: integer (nullable = true)
 |-- trip_month: i

# **Evaluación de modelo**

In [10]:
# =============================================================================
# Selección avanzada de funciones
# =============================================================================

from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
import pandas as pd

# --- Paso 1: Preparación de datos ---

# Realizar un muestreo estratificado en la columna de cadena original.
# Primero, buscar los distintos tipos de usuario para crear el diccionario de fracciones.
label_col = 'member_casual'
distinct_labels = [row[label_col] for row in df_final.select(label_col).distinct().collect()]
fractions = {label: 0.5 for label in distinct_labels} # % de muestra para cada tipo de usuario

# Crear la muestra estratificada y divídala en conjuntos de entrenamiento y prueba
df_sample = df_final.sampleBy(label_col, fractions=fractions, seed=42)
train_data, test_data = df_sample.randomSplit([0.8, 0.2], seed=42)

# Almacenar en caché los datos de entrenamiento para un acceso más rápido durante el entrenamiento del modelo
train_data.cache().count()

# --- Paso 2: Definir el pipeline completo ---

# El pipeline ahora gestionará todas las transformaciones, incluida la indexación de etiquetas.
# Esto garantiza que se ajuste a los datos que aún no tienen una columna de "label".

# 1. Indexar la columna de etiquetas
label_indexer = StringIndexer(inputCol=label_col, outputCol='label', handleInvalid='keep')

# 2. Definir características categóricas y numéricas
categorical_features = ['season']
numerical_features = [
    'log_duration', 'haversine_distance_km', 'is_popular_start',
    'trip_hour', 'trip_day_of_week', 'trip_month',
    'hour_sin', 'hour_cos', 'day_of_week_sin', 'day_of_week_cos'
]

# 3. Crear etapas para la codificación de características categóricas
stages = [label_indexer]
encoded_feature_names = []
for c in categorical_features:
    indexer = StringIndexer(inputCol=c, outputCol=f"{c}_indexed", handleInvalid='keep')
    encoder = OneHotEncoder(inputCol=f"{c}_indexed", outputCol=f"{c}_encoded", dropLast=False)
    stages.extend([indexer, encoder])
    encoded_feature_names.append(f"{c}_encoded")

# 4. Ensamblar todas las características en un solo vector
all_assembler_cols = numerical_features + encoded_feature_names
assembler = VectorAssembler(inputCols=all_assembler_cols, outputCol="features")
stages.append(assembler)

# 5. Definir el modelo RandomForest para obtener la importancia de las características
rf = RandomForestClassifier(labelCol='label', featuresCol="features", numTrees=100, seed=42)
stages.append(rf)

# Crear el objeto de pipeline completo
pipeline = Pipeline(stages=stages)

# --- Paso 3: Entrenar el modelo y extraer la importancia de las características ---

# Entrenar el modelo de pipeline con los datos de entrenamiento sin procesar (no indexados).
# Esta única llamada a '.fit()' ejecuta correctamente todas las etapas en orden.
model = pipeline.fit(train_data)

# Extraer puntuaciones de importancia de la etapa del modelo RandomForest
importances = model.stages[-1].featureImportances.toArray()

# Obtener los nombres de las características de la etapa del ensamblador en el modelo al que se aplico '.fit()'
feature_names = model.stages[-2].getInputCols()

# Crear un DataFrame para mostrar la importancia de las características
importance_df = pd.DataFrame(list(zip(feature_names, importances)), columns=['feature', 'importance'])
importance_df = importance_df.sort_values('importance', ascending=False)

print("Las 10 características principales por importancia:")
print(importance_df.head(10))

# Seleccionar dinámicamente las 10 características principales para el modelo final
top_features = importance_df.head(10)['feature'].tolist()
print(f"\nCaracterísticas seleccionadas para el modelo final:\n{top_features}")

Las 10 características principales por importancia:
                  feature  importance
0            log_duration    0.453051
9         day_of_week_cos    0.189940
1   haversine_distance_km    0.147129
4        trip_day_of_week    0.103232
6                hour_sin    0.025357
10         season_encoded    0.015699
5              trip_month    0.010721
3               trip_hour    0.007126
8         day_of_week_sin    0.006529
2        is_popular_start    0.006483

Características seleccionadas para el modelo final:
 ['log_duration', 'day_of_week_cos', 'haversine_distance_km', 'trip_day_of_week', 'hour_sin', 'season_encoded', 'trip_month', 'trip_hour', 'day_of_week_sin', 'is_popular_start']


In [9]:
# =============================================================================
# Reevaluación del modelo con características seleccionadas
# =============================================================================
# Este bloque utiliza las características principales para entrenar y evaluar un modelo de regresión logística final más eficiente.
# Ahora separa correctamente la transformación de características del ajuste del modelo para corregir el error.

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, PipelineModel
from pyspark.mllib.evaluation import MulticlassMetrics
import pandas as pd

# --- Paso 1: Preparar datos utilizando transformadores del primer pipeline ---

# El primer pipeline ('modelo') contiene todos los transformadores de características necesarios (indexadores, codificadores).
# Se creará un nuevo PipelineModel que contenga SOLO estas etapas de características, excluyendo la etapa final de RandomForestClassifier.
# model.stages[:-1] selecciona todas las etapas excepto la última.
feature_transformer_pipeline = PipelineModel(stages=model.stages[:-1])

# Ahora, se transforma los datos de entrenamiento y prueba para crear todas las columnas de características diseñadas
# sin agregar una columna de "predicción".
transformed_train_data = feature_transformer_pipeline.transform(train_data)
transformed_test_data = feature_transformer_pipeline.transform(test_data)

# --- Paso 2: Construcción dinámica del pipeline para el modelo final ---

print(f"Reconstruyendo el pipeline con las siguientes características:\n {top_features}\n")

# Identificar dinámicamente las características finales para el ensamblador.
# La lista 'top_features' ya contiene los nombres correctos de las columnas codificadas.
final_assembler_cols = top_features

# Comenzar a construir el pipeline final.
# NOTA: NO necesita un indexador de etiquetas.
# El modelo de regresión logística funcionará en las columnas "label" y "final_features".
final_pipeline_stages = []

# 1. Ensamblar solo las características principales seleccionadas en un nuevo vector de características
assembler_final = VectorAssembler(inputCols=final_assembler_cols, outputCol="final_features")
final_pipeline_stages.append(assembler_final)

# 2. Definir el modelo final (Regresión logística)
lr = LogisticRegression(labelCol="label", featuresCol="final_features", maxIter=100)
final_pipeline_stages.append(lr)

# Crear el objeto pipeline_final
pipeline_final = Pipeline(stages=final_pipeline_stages)

# --- Paso 3: Entrenamiento y evaluación de modelos ---

# Entrenar el nuevo pipeline, más enfocado, en los datos de entrenamiento transformados
final_model = pipeline_final.fit(transformed_train_data)

# Realizar predicciones sobre el conjunto de pruebas transformado
predictions = final_model.transform(transformed_test_data)

# --- Paso 4: Métricas de rendimiento ---

evaluator_accuracy = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator_accuracy.evaluate(predictions)

print(f"Nueva precisión del modelo: {accuracy:.4f}")
print(f"Precisión de referencia: 0.7524")
improvement = accuracy - 0.7524
print(f"Mejora del modelo: {improvement:+.4f}\n")

# MulticlassMetrics para la matriz de confusión y el informe de clasificación
predictionAndLabels = predictions.select('prediction', 'label').rdd.map(lambda r: (float(r.prediction), float(r.label)))
metrics = MulticlassMetrics(predictionAndLabels)

# Matriz de confusión
confusion_matrix = metrics.confusionMatrix().toArray()
class_labels = model.stages[0].labels # Obtener etiquetas de clase del primer pipeline
report_df = pd.DataFrame(confusion_matrix,
                         index=[f'Actual: {l}' for l in class_labels],
                         columns=[f'Predicted: {l}' for l in class_labels])

print("Matriz de confusión:")
print(report_df)

# Determinar la etiqueta de clase positiva para el informe
label_map = {i: l for i, l in enumerate(class_labels)}
positive_class_label = 1.0 if label_map.get(1.0) == 'member' else 0.0

print("\nInforme de clasificación (para la clase 'member'):")
print(f"Precision: {metrics.precision(positive_class_label):.4f}")
print(f"Recall: {metrics.recall(positive_class_label):.4f}")
print(f"F1-Score: {metrics.fMeasure(positive_class_label):.4f}\n")

# Desconservar los datos almacenados en caché
train_data.unpersist()

Reconstruyendo el pipeline con las siguientes características:
 ['log_duration', 'day_of_week_cos', 'haversine_distance_km', 'trip_day_of_week', 'hour_sin', 'season_encoded', 'trip_hour', 'hour_cos', 'is_popular_start', 'day_of_week_sin']

Nueva precisión del modelo: 0.7856
Precisión de referencia: 0.7524
Mejora del modelo: +0.0332



/usr/local/lib/python3.11/dist-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Matriz de confusión:
                Predicted: member  Predicted: casual
Actual: member           374182.0             9046.0
Actual: casual            96227.0            11558.0

Informe de clasificación (para la clase 'member'):
Precision: 0.7954
Recall: 0.9764
F1-Score: 0.8767



DataFrame[start_station_id: string, ride_id: string, started_at: timestamp, ended_at: timestamp, start_station_name: string, start_lat: double, start_lng: double, end_station_id: string, end_station_name: string, end_lat: double, end_lng: double, member_casual: string, duration_sec: bigint, schema_version: string, periodo: string, log_duration: double, trip_year: int, trip_month: int, trip_day_of_week: int, trip_hour: int, is_weekend: int, season: string, hour_sin: double, hour_cos: double, day_of_week_sin: double, day_of_week_cos: double, haversine_distance_km: double, is_popular_start: int]